In [1]:
import torch
import torch.nn as nn
import torch.utils.data as Data
import torch.optim as optim
from torchvision import transforms  # Import the transforms module for data preprocessing
import matplotlib.pyplot as plt
import numpy as np
import os  # Import the os module to operate the file path
from PIL import Image  # Import the Image module in the PIL library for image processing
from tqdm import tqdm  # Progress bar module
from time import sleep 
import time # Used to record training time

In [2]:
# Check if CUDA is installed and available for your GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
# 设置随机数种子
def setup_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    # random.seed(seed)
    torch.backends.cudnn.deterministic = True

SEED = 2809
setup_seed(SEED)

### To load Dataset

In [4]:
# The class for loading datasets is used to load datasets 
# whose labels are stored in a separate txt file in the format of "filename\t label"
class CustomDataset(Data.Dataset):
    def __init__(self, image_dir, label_file, transform=None):
        super().__init__()  # Calling the parent class's constructor
        self.image_dir = image_dir  # Path to the image data
        self.label_file = label_file  # The path of the label text
        self.transform = transform  # Data preprocessing operations
        self.samples = self._load_samples()  # Loading dataset sample information

    def _load_samples(self):
        samples = []  # List storing sample information
        with open(self.label_file, 'r') as f:  # Open the label text file
            for line in f:  # Read the contents of a labeled text file line by line
                image_name, label = line.strip().split('\t ')  # Get the image file name and label by separating each line with tabs + spaces
                image_path = os.path.join(self.image_dir, image_name)  # The full path to the stitched image file
                samples.append((image_path, label))  # Combine the image path and label into a tuple and add it to the sample list
        return samples  # Return to sample list

    def __len__(self):
        return len(self.samples)  # Returns the number of samples in the dataset

    def __getitem__(self, index):
        image_path, label = self.samples[index]  # Get the image path and label at the specified index
        image = Image.open(image_path).convert('L')  # Open an image file and convert it to grayscale
        if self.transform:  # If data preprocessing operations are defined
            image = self.transform(image)  # Preprocess the image
        return image, label  # Returns the preprocessed image and label

In [5]:
# Set the image data path and label text path
train_image_dir = '../data_train/graphs/'  # Path to the image data
train_label_file = '../data_train/labels.txt'  # The path of the label text
test_image_dir = '../data_test/graphs/' 
test_label_file = '../data_test/labels.txt'

# Define data preprocessing operations and add other preprocessing operations as needed
transform = transforms.Compose([
    transforms.Resize((40, 40)),  # Resize the images
    transforms.ToTensor(),  # Convert image to tensor
])

# Creating a custom dataset instance
train_dataset = CustomDataset(train_image_dir, train_label_file, transform=transform)
test_dataset = CustomDataset(test_image_dir, test_label_file, transform=transform)

print("Size of train dataset:")
print(train_dataset.__len__())
print("Size of test dataset:")
print(test_dataset.__len__())

# Creating a Data Loader
# Shuffle the order during training
train_loader = Data.DataLoader(dataset=train_dataset, batch_size=1, shuffle=False)
test_loader = Data.DataLoader(dataset=test_dataset, batch_size=1, shuffle=False)

Size of train dataset:
18000
Size of test dataset:
600


### To load the model

In [6]:
class AutoEncoder(nn.Module):
    def __init__(self):
        super(AutoEncoder, self).__init__()

        self.encoder = nn.Sequential(
            nn.Linear(in_features=40*40, out_features=60),
        )
        self.decoder = nn.Sequential(
            nn.Linear(in_features=60, out_features=40*40),
            nn.Sigmoid(),       # compress to a range (0, 1)
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [7]:
new_model = AutoEncoder().to(device)
new_model.load_state_dict(torch.load("./model_param/AE_MSE_ReLU_0.01_60.pth"))

<All keys matched successfully>

### To make new dataset

In [8]:
with open("./dataset/labels_train.txt", 'w') as file: # 打开文件
    i=1
    for data in tqdm(train_loader):
        img, label = data
        img = img.to(device)
        img = img.flatten(2, 3)
        # Forward Propagation
        output = new_model.encoder(img)
        torch.save(output.clone(), "./dataset/data_train/tensor"+str(i)+".pth")
        file.write("tensor"+str(i)+".pth\t "+label[0]+'\n') # 将内容写到文件中
        i+=1
file.close() # 关闭文件，此处可以理解为保存

100%|███████████████████████████████████████████████████████████████████████████| 18000/18000 [02:28<00:00, 121.21it/s]


In [9]:
with open("./dataset/labels_test.txt", 'w') as file: # 打开文件
    i=1
    for data in tqdm(test_loader):
        img, label = data
        img = img.to(device)
        img = img.flatten(2, 3)
        # Forward Propagation
        output = new_model.encoder(img)
        torch.save(output.clone(), "./dataset/data_test/tensor"+str(i)+".pth")
        file.write("tensor"+str(i)+".pth\t "+label[0]+'\n') # 将内容写到文件中
        i+=1
file.close() # 关闭文件，此处可以理解为保存

100%|███████████████████████████████████████████████████████████████████████████████| 600/600 [00:05<00:00, 116.54it/s]
